# Document Intelligence Service 
Extract information from documents using a Custom Trained Model In Document Intelligence Service

## Prerequisites
1. To run the code, install the following packages. Please use the latest pre-release version `pip install azure-ai-documentintelligence`.


In [3]:
"""
This code sample shows Custom Extraction Model operations with the Azure AI Document Intelligence client library.
The async versions of the samples require Python 3.8 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
import os

endpoint = os.getenv('DOCINTEL_API_ENDPOINT')
key = os.getenv('DOCINTEL_API_KEY')

#model_id = "autoInsuranceClaimExtraction-v2"
#formUrl = f"{os.getenv('TR_BLOB_STORE_GENERAL')}/docai-samples/AutoInsuranceClaims/IC-handwritten-JeanGenet.pdf"
model_id = "commercialInsuranceApplicationExtraction-v1"
formUrl = f"{os.getenv('TR_BLOB_STORE_GENERAL')}/docai-samples/CommercialInsuranceApplications/ABCChemicals-NECommercialConstructionCompany-COMMERCIAL INSURANCE APPLICATION.pdf"
#model_id = "workersCompensationApplicationExtraction-v1"
#formUrl = f"{os.getenv('TR_BLOB_STORE_GENERAL')}/docai-samples/WorkerCompensation/WilliamWise - WORKERS COMPENSATION APPLICATION.pdf"


document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Make sure your document's type is included in the list of document types the custom model can analyze
poller = document_intelligence_client.begin_analyze_document(
    model_id, AnalyzeDocumentRequest(url_source=formUrl)
)
result = poller.result()

for idx, document in enumerate(result.documents):
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has confidence {}".format(document.confidence))
    print("Document was analyzed by model with ID {}".format(result.model_id))
    for name, field in document.fields.items():
        print("......found field of type '{}' with value '{}' and with confidence {}".format(field.type, field.content, field.confidence))


# iterate over tables, lines, and selection marks on each page
for page in result.pages:
    print("\nLines found on page {}".format(page.page_number))
    for line in page.lines:
        print("...Line '{}'".format(line.content.encode('utf-8')))
    for word in page.words:
        print(
            "...Word '{}' has a confidence of {}".format(
                word.content.encode('utf-8'), word.confidence
            )
        )
    if page.selection_marks:
        for selection_mark in page.selection_marks:
            print(
                "...Selection mark is '{}' and has a confidence of {}".format(
                    selection_mark.state, selection_mark.confidence
                )
            )

for i, table in enumerate(result.tables):
    print("\nTable {} can be found on page:".format(i + 1))
    for region in table.bounding_regions:
        print("...{}".format(i + 1, region.page_number))
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index, cell.column_index, cell.content.encode('utf-8')
            )
        )
print("-----------------------------------")


--------Analyzing document #1--------
Document has type commercialInsuranceApplicationExtraction-v1
Document has confidence 0.999
Document was analyzed by model with ID commercialInsuranceApplicationExtraction-v1
......found field of type 'DocumentFieldType.STRING' with value 'COMMERCIAL INSURANCE APPLICATION APPLICANT INFORMATION SECTION' and with confidence 0.943
......found field of type 'DocumentFieldType.STRING' with value '11/20/2023' and with confidence 0.983
......found field of type 'DocumentFieldType.STRING' with value 'International Insurers Inc' and with confidence 0.878
......found field of type 'DocumentFieldType.STRING' with value '602 5th Ave
Asbury Park
NJ 07712
NJ USA' and with confidence 0.92
......found field of type 'DocumentFieldType.STRING' with value 'GCML' and with confidence 0.974
......found field of type 'DocumentFieldType.STRING' with value '123' and with confidence 0.724
......found field of type 'DocumentFieldType.STRING' with value 'ABC Chemicals LLC' an